In [37]:
import pandas as pd
import tensorflow as tf
import numpy as np
import sklearn as sk

import functools

###  Data Information

#### Energy boost Data
Data is a file containing a sequence of keys that show if a sequence of songs leads to an energy boost, The labels y mark energy boosts with a 1 and no energ boost with a 0. For songs to also be categorized as energy boosts the tempo can not decrease more than 2% and can not increase more than 4%

In [55]:
# Loading Data

file_name = 'C:\\Users\\roike\\Documents\\research\\MachineDJ\\DataSets\\energy_boost_xy.xlsx'
mix_df = pd.read_excel(file_name)

num_of_keys = 12
convert_dict = {
                "tempo1": np.float32,
                "tempo2": np.float32,
                "y": np.int16
               }

for i in range(num_of_keys):
    dict_key = "key1"+str(i+1)
    convert_dict[dict_key] = np.int32

for i in range(num_of_keys):
    dict_key = "key2"+str(i+1)
    convert_dict[dict_key] = np.int32

mix_df = mix_df.astype(convert_dict)

In [56]:
mix_df.dtypes

mode1       int64
tempo1    float32
key11       int32
key12       int32
key13       int32
key14       int32
key15       int32
key16       int32
key17       int32
key18       int32
key19       int32
key110      int32
key111      int32
key112      int32
mode2       int64
tempo2    float32
key21       int32
key22       int32
key23       int32
key24       int32
key25       int32
key26       int32
key27       int32
key28       int32
key29       int32
key210      int32
key211      int32
key212      int32
y           int16
dtype: object

In [63]:
df = mix_df.copy()

In [64]:
target = df.pop("y")

In [65]:
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))

In [66]:
for feat, targ in dataset.take(5):
    print ("Features: {}, target: {}".format(feat, targ))

Features: [1.         0.34188035 0.         0.         1.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         1.         0.34188035 0.         0.
 1.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ], target: 0
Features: [1.         0.34188035 0.         0.         1.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         1.         0.34188035 1.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ], target: 0
Features: [1.         0.34188035 0.         0.         1.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         1.         0.34188035 0.         1.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ], target: 0
Features: [1.         0.34188035 0.         0.         1.         0.
 0.         0.        

In [67]:
print(list(dataset.take(1)))

[(<tf.Tensor: shape=(28,), dtype=float64, numpy=
array([1.        , 0.34188035, 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 1.        ,
       0.34188035, 0.        , 0.        , 1.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        ])>, <tf.Tensor: shape=(), dtype=int16, numpy=0>)]


In [72]:
DATASIZE = 1000

test_size = int(DATASIZE * 0.7)
val_size = int(DATASIZE * 0.15)
test_size = int(DATASIZE * 0.15)

dataset = dataset.shuffle(100*DATASIZE)
train_dataset = dataset.take(test_size)
test_dataset = dataset.skip(test_size)
val_dataset = test_dataset.take(val_size)
test_dataset = test_dataset.skip(val_size)

In [96]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Flatten


In [81]:
 list(train_dataset.take(1) )
# print("Training data size : {}".x_train)

[(<tf.Tensor: shape=(28,), dtype=float64, numpy=
  array([0.        , 0.38461539, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 1.        , 0.        ,
         0.32478634, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 1.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ])>,
  <tf.Tensor: shape=(), dtype=int16, numpy=0>)]

In [88]:
feat_num = 28


In [112]:
model = Sequential()
# model.add(Flatten())
model.add(LSTM(128, input_shape=(1,feat_num), activation='relu', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(128, activation='relu'))
model.add(Dropout(0.1))

# model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

# model.add(Flatten())
model.add(Dense(10, activation='softmax'))

In [113]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

In [115]:
model.fit(train_dataset, epochs=3,validation_data=val_dataset)

NameError: name 'X' is not defined